<a href="https://colab.research.google.com/github/D-2000-99/Tensorflow-Object-Detection/blob/main/Tensorflow_Custom_Object_Detection_on_Colab_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

- This is a template for implementing fine-tuning of an object detection model on Colab. The training process can be extensive as it takes a long time to implement so this notebook atempts to abstract away the mundane tasks to allow quicker implementation and training on colab.Using the GPU runtime will be ESSENTIAL.  
- You are required to follow the [TensorFlow 2 Object Detection API tutorial](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html) along with this notebook.

#Imports

## Cloning TF Model Repo

In [ ]:
#Cloning TF repo with all models
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 68232, done.
remote: Total 68232 (delta 0), reused 0 (delta 0), pack-reused 68232
Receiving objects: 100% (68232/68232), 576.51 MiB | 27.26 MiB/s, done.
Resolving deltas: 100% (47944/47944), done.


##Protobuf

In [ ]:
#Preinstalled in Colab
!cd models/research/ && protoc object_detection/protos/*.proto --python_out=.

## Installing Object Detection API

In [ ]:
#Running models/research/object_detection/packages/tf2/setup.py
!cd models/research && \
  cp object_detection/packages/tf2/setup.py . && \
  python -m pip install .

Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1683460 sha256=bf15c8ffd51e2c0bfb5709f40bb1389d54226b7b0702a5c73c8e01594e7f69f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-you0ho2k/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


##Importing base libraries

In [ ]:
#File/Image handling
import os
from PIL import Image
import imageio #For video
import pathlib #For Path
import time #measure processing time

#Essential DL packages
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

#Object-Detection packages 
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils #For annotation of Video

In [ ]:
# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
#Ensure GPU's are enabled to prevent extremely slow training
gpus

[]

In [ ]:
!mkdir Tensorflow && mv models Tensorflow

mkdir: cannot create directory ‘Tensorflow’: File exists


##Pycocotools
Cloning pycocotools and copying to Tensorflow/models/research/

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI && make && cp -r pycocotools /content/Tensorflow/models/research/

fatal: destination path 'cocoapi' already exists and is not an empty directory.
python setup.py build_ext --inplace
running build_ext
skipping 'pycocotools/_mask.c' Cython extension (up-to-date)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I../common -I/usr/include/python3.7m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.7/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
../common/maskApi.c: In function ‘rleDecode’:
../commo

Run optional tester code to confirm the code working

In [ ]:
#Tester Script
#!python /content/Tensorflow/models/research/object_detection/builders/model_builder_tf2_test.py

##Creating folder structure - Workspace

workspace/  
│  ├─ training_demo/  
│  ├─ annotations/  
│  ├─ exported-models/  
│  ├─ images/  
│  │  ├─ test/  
│  │  ├─ train/  
│  ├─ models/  
│  ├─ pre-trained-models/  
│  ├─ README.md

In [ ]:
!cd /content/Tensorflow && mkdir workspace
!cd /content/Tensorflow/workspace && mkdir training_demo

mkdir: cannot create directory ‘workspace’: File exists
mkdir: cannot create directory ‘training_demo’: File exists


In [ ]:
!cd /content/Tensorflow/workspace/training_demo &&\
mkdir annotations &&\
mkdir exported-models &&\
mkdir images &&\
mkdir models &&\
mkdir pre-trained-models

mkdir: cannot create directory ‘annotations’: File exists


In [ ]:
!cd /content/Tensorflow/workspace/training_demo/images &&\
mkdir train &&\
mkdir test

mkdir: cannot create directory ‘train’: File exists


In [ ]:
!cd /content/Tensorflow/ && mkdir scripts
!cd /content/Tensorflow/scripts/ && mkdir preprocessing

mkdir: cannot create directory ‘scripts’: File exists
mkdir: cannot create directory ‘preprocessing’: File exists


---
##Run everything before this by default
---

#Preparing the Dataset

Steps:
1. Annotate the Dataset on your computer with labelImg [Link](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#annotate-the-dataset)
2. Upload the images to train and test respectively.   
  -->**Train:** Tensorflow\workspace\training_demo\images\train  
  -->**Test:** Tensorflow\workspace\training_demo\images\test  
3. Create a label map and upload to workspace/annotations
4. Upload generate_tfrecord.py to Tensorflow/scripts/preprocessing [Link](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records) (Scroll down till you see the download option)
5. Do not just run everything after this, you will be required to select your model and upload your custom pipeline.config or you may run into issues.

##Creating TFRecords

In [ ]:
#Training
!cd /content/Tensorflow/scripts/preprocessing/ &&\
python generate_tfrecord.py -x /content/Tensorflow/workspace/training_demo/images/train -l /content/Tensorflow/workspace/training_demo/annotations/label_map.pbtxt -o /content/Tensorflow/workspace/training_demo/annotations/train.record

2021-12-28 13:44:09.601639: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Successfully created the TFRecord file: /content/Tensorflow/workspace/training_demo/annotations/train.record


In [ ]:
#Test
!cd /content/Tensorflow/scripts/preprocessing/ &&\
python generate_tfrecord.py -x /content/Tensorflow/workspace/training_demo/images/test -l /content/Tensorflow/workspace/training_demo/annotations/label_map.pbtxt -o /content/Tensorflow/workspace/training_demo/annotations/test.record

2021-12-28 13:44:27.506035: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Successfully created the TFRecord file: /content/Tensorflow/workspace/training_demo/annotations/test.record


#Downloading Model from TFOD Zoo

Edit the url below to whatever model you want to use.  
NOTE: Does not work with all models eg. tflite models such as MobileNet WILL NOT WORK.

In [ ]:
#Change to required model from model zoo
url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz' #Change here
PATH_TO_MODEL_DIR = tf.keras.utils.get_file(
    fname='ssd_resnet50_v1_fpn_640x640_coco17_tpu-8', # And here (Usually the last part minus the .tar.gz)
    origin=url,
    untar=True
)

244826112/244817203 [==============================] - 2s 0us/step


In [ ]:
!cp {PATH_TO_MODEL_DIR} /content/Tensorflow/workspace/training_demo/pre-trained-models -r

---

#Pipeline.config

In [ ]:
#Copying pipeline.config to models/my_ssd_resnet50_v1_fpn
!mkdir /content/Tensorflow/models/my_ssd_resnet50_v1_fpn #Creating folder
!cp /content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config\
 /content/Tensorflow/models/my_ssd_resnet50_v1_fpn #Copying pipeline.config 

mkdir: cannot create directory ‘/content/Tensorflow/models/my_ssd_resnet50_v1_fpn’: File exists


**Important Step:**  
A Default pipeline.config is copied which has WAY too many steps and too big a batch size for a free tier. 

Upload your configured pipeline.config now to --> Tensorflow/models/my_ssd_resnet50_v1_fpn

---

In [ ]:
|#Copying TensorFlow/models/research/object_detection/model_main_tf2.py to Tensorflow/workspace/training_demo folder
!cp /content/Tensorflow/models/research/object_detection/model_main_tf2.py\
 /content/Tensorflow/workspace/training_demo

#Training the model

Here is where a lot people face issues, on the free Colab tier GPU with a batch size of 2 and around 4000 steps, it took around 30 mins to train my model on 20 images. As mentioned in the API tutorial it may seem frozen but it only outputs every 100 steps, so be patient!

MAKE SURE YOUR RUNTIME HAS A GPU !!!!!!!!!!!

In [ ]:
!cd /content/Tensorflow/workspace/training_demo &&\
python model_main_tf2.py --model_dir=../../models/my_ssd_resnet50_v1_fpn/ --pipeline_config_path=../../models/my_ssd_resnet50_v1_fpn/pipeline.config

2021-12-28 14:16:51.255818: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
W1228 14:16:51.257413 140242319992704 cross_device_ops.py:1387] There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
I1228 14:16:51.258694 140242319992704 mirrored_strategy.py:376] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I1228 14:16:51.265101 140242319992704 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1228 14:16:51.265300 140242319992704 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W1228 14:16:51.318756 140242319992704 deprecation.py:347] From /usr/local/

In [ ]:
!cd /content/Tensorflow/workspace/training_demo &&\
 ls ../../models/my_ssd_resnet50_v1_fpn/

pipeline.config


Now model is ready, you can zip up the model from it -->   Tensorflow\workspace\training_demo\exported-models\my_model  
  
Sample given below:

In [ ]:
#!zip -r /content/final_model.zip Tensorflow\workspace\training_demo\exported-models\my_model

That's it you can now use your custom model for Object detection code! Happy detection :)